# <span style="color: #4CAF50; font-family: 'Arial, sans-serif';">LP1 PROJECT - Team Silicon</span>
## <span style="color: #2196F3; font-family: 'Verdana, sans-serif';">AZUBI AFRICA PROJECT</span>

# INTRODUCTION
##### As data analyst, we will explore the Indian startup ecosystem by analyzing data spanning from 2018 to 2021. This project will adhere to the CRISP-DM framework, aiming to deliver meaningful insights to our client.










--

--

# Business Understanding:
### ***Identify Key Trends:***

##### Objective: 
Uncover and understand the key trends in the Indian startup ecosystem over the four-year period (2018-2021).
##### Goals: 
Identify sectors experiencing the most significant growth, the distribution of funding across rounds, and the geographic distribution of startup activities.

### ***Evaluate Funding Patterns:***
##### Objective: 
Analyze funding patterns to understand how different sectors, rounds, and company activities attract investment.
##### Goals: 
Identify sectors receiving the highest and lowest funding, explore the distribution of funding amounts, and understand the most common types of company activities that attract investment.

### ***Assess Investor Involvement:***
##### Objective: 
Understand the role and impact of investors in the Indian startup ecosystem.
##### Goals:
 Identify the most active investors, assess their investment patterns, and understand the correlation between investor involvement and 
startup success.

# Project Goals:

### ***Clean and Prepare Data:***

##### Objective: 
Ensure the dataset is cleaned and prepared for analysis.
##### Goals: 
Handle missing values, standardize data types, and address any inconsistencies or outliers.

### ***Perform Exploratory Data Analysis (EDA):***
##### Objective: 
Gain insights through exploratory analysis.
##### Goals: 
Visualize trends over time, explore distributions of funding amounts, and identify notable patterns in the data.

### ***Communicate Findings:***
##### Objective: 
Effectively communicate insights and findings.
##### Goals: 
Create clear and concise visualizations, reports, and presentations to convey key findings to stakeholders.

### ***Provide Recommendations:***
##### Objective: 
Offer actionable recommendations based on the analysis.
##### Goals: 
Suggest strategies for startups, investors, and policymakers based on identified trends and opportunities.

# HYPOTHESIS TESTING 
#### Null Hypothesis (HO) - The amount of funding for companies is not influenced by the company's sector.
#### Alternate Hypothesis (H1) - The amount of funding for companies is influenced by the company's sector.

# Business Questions
### 1. Does the sector in which a company operates significantly influence its funding amount?
### 2. Are there observable differences in funding amounts based on the industry sector?
### 3. Can we identify specific sectors that tend to receive higher or lower funding amounts?
### 4. Is there a relationship between the company's sector and the variability in funding received?

--

# PROJECT WORK

In [3]:
# Importing the necessary libraries
import pyodbc     
#import the dotenv_values function from the dotenv package
from dotenv import dotenv_values    
import pandas as pd
import warnings 
import re

warnings.filterwarnings('ignore')

In [4]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("server")
database = environment_variables.get("database")
username = environment_variables.get("username")
password = environment_variables.get("password")

In [5]:
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

In [6]:
# Use the connect method of the pyodbc library and pass in the connection string.

connection = pyodbc.connect(connection_string)

In [7]:
# importing the data sets to the environment. The first two are SQL queries to pull the 2020 & 2021 data from the server
# The data3 & 4 are downloaded data sets thus importing them using pandas read_csv() for the 2018 & 2019 data sets

query1 = "SELECT * FROM dbo.LP1_startup_funding2021"
data1 = pd.read_sql(query1, connection)
query2 = "SELECT * FROM dbo.LP1_startup_funding2020"
data2 = pd.read_sql(query2, connection)
data3 = pd.read_csv("startup_funding2018.csv")
data4 =  pd.read_csv("startup_funding2019.csv")


### Checking Data sets loaded on to notebook

In [8]:
data1.head(2)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None


In [9]:
data2.head(2)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None


In [10]:
data3.head(2)

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...


In [11]:
data4.head(2)

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C


### Concating the datasets to one Data frame  'df'

In [12]:
df = pd.concat([data1, data2, data3, data4], ignore_index = True)

In [13]:
# Checking the concatenated data-frame
df.head(2)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10,Company Name,Industry,Round/Series,Location,About Company,Company/Brand,What it does,Amount($)
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  2264 non-null   object 
 1   Founded        2110 non-null   float64
 2   HeadQuarter    2239 non-null   object 
 3   Sector         2335 non-null   object 
 4   What_it_does   2264 non-null   object 
 5   Founders       2334 non-null   object 
 6   Investor       2253 non-null   object 
 7   Amount         2533 non-null   object 
 8   Stage          1415 non-null   object 
 9   column10       2 non-null      object 
 10  Company Name   526 non-null    object 
 11  Industry       526 non-null    object 
 12  Round/Series   526 non-null    object 
 13  Location       526 non-null    object 
 14  About Company  526 non-null    object 
 15  Company/Brand  89 non-null     object 
 16  What it does   89 non-null     object 
 17  Amount($)      89 non-null     object 
dtypes: float

#### The above output gives a summary of the concatenated data-frame structure, including column names, non-null counts, data types, and memory usage. 

### Combining similar columns in the Concatenated df and droping the unecessary columns  

In [15]:
#Merging columns with similar data to make a cleaner dataset
df['company_name'] = df['Company_Brand'].fillna(df['Company Name']).fillna(df['Company/Brand'])
df['amount'] = df['Amount'].fillna(df['Amount($)'])
df['company_activity'] = df['What_it_does'].fillna(df['What it does']).fillna(df['About Company'])
df['sector'] = df['Sector'].fillna(df['Industry'])
df['round'] = df['Stage'].fillna(df['Round/Series'])
df['head_quarter'] = df['HeadQuarter'].fillna(df['Location'])
df['year_founded'] = df['Founded']
df['founders'] = df['Founders']
df['investor'] = df['Investor']

# Droping the original columns 
df = df.drop(['Company_Brand', 'Company Name', 'Company/Brand', 'Amount', 'Amount($)', 'What_it_does', 'What it does','About Company',
              'column10', 'Sector', 'Industry','Stage', 'Round/Series', 'HeadQuarter', 'Location', 'Founded', 'Founders', 'Investor'], axis=1)

### Checking new data-frame properties and column data types

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company_name      2879 non-null   object 
 1   amount            2622 non-null   object 
 2   company_activity  2879 non-null   object 
 3   sector            2861 non-null   object 
 4   round             1941 non-null   object 
 5   head_quarter      2765 non-null   object 
 6   year_founded      2110 non-null   float64
 7   founders          2334 non-null   object 
 8   investor          2253 non-null   object 
dtypes: float64(1), object(8)
memory usage: 202.6+ KB


### Checking Data-frame description 

In [17]:
df.describe(include = 'all')

,company_name,amount,company_activity,sector,round,head_quarter,year_founded,founders,investor
count,2879,2622,2879,2861,1941,2765,2110.000000,2334,2253
unique,2214,774,2691,873,75,172,NaN,1980,1777
top,BharatPe,—,Provides online learning classes,FinTech,Seed,Bangalore,NaN,"Ashneer Grover, Shashvat Nakrani",Inflection Point Ventures
freq,10,148,5,173,606,764,NaN,7,36
mean,NaN,NaN,NaN,NaN,NaN,NaN,2016.079621,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,4.368006,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1963.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2015.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,2017.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,2019.000000,NaN,NaN


In [18]:

# Convert 'year_founded' column to datetime
df['year_founded'] = pd.to_datetime(df['year_founded'], errors='coerce', format='%Y')

In [20]:
# This code uses a regular expression to find all non-digit, non-comma, and non-dot characters in each cell of the 'amounts' column.
symbols = df['amount'].apply(lambda x: re.findall(r'[^\d.,]+', str(x)))
unique_symbols = set(symbol for symbols_list in symbols for symbol in symbols_list)
unique_symbols

{' LetsVenture',
 '$',
 '$$',
 '$Undisclosed',
 '$undisclosed',
 'ITO Angel Network',
 'JITO Angel Network',
 'Pre-series A',
 'Seed',
 'Series C',
 'Undisclosed',
 'Upsparks',
 'ah! Ventures',
 'nan',
 '—',
 '₹'}

In [ ]:
# Checking data-frame info to see if column data types are correct
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   company_name      2879 non-null   object        
 1   amount            2622 non-null   object        
 2   company_activity  2879 non-null   object        
 3   sector            2861 non-null   object        
 4   round             1941 non-null   object        
 5   head_quarter      2765 non-null   object        
 6   year_founded      2110 non-null   datetime64[ns]
 7   founders          2334 non-null   object        
 8   investor          2253 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 202.6+ KB


In [ ]:
# Checking the new data-frame
df.tail()

,company_name,amount,company_activity,sector,round,head_quarter,year_founded,founders,investor
2874,Infra.Market,"$20,000,000",It connects client requirements to their suppl...,Infratech,Series A,Mumbai,NaT,"Aaditya Sharda, Souvik Sengupta","Tiger Global, Nexus Venture Partners, Accel Pa..."
2875,Oyo,"$693,000,000",Provides rooms for comfortable stay,Hospitality,NaN,Gurugram,2013-01-01,Ritesh Agarwal,"MyPreferred Transformation, Avendus Finance, S..."
2876,GoMechanic,"$5,000,000",Find automobile repair and maintenance service...,Automobile & Technology,Series B,Delhi,2016-01-01,"Amit Bhasin, Kushal Karwa, Nitin Rana, Rishabh...",Sequoia Capital
2877,Spinny,"$50,000,000",Online car retailer,Automobile,NaN,Delhi,2015-01-01,"Niraj Singh, Ramanshu Mahaur, Ganesh Pawar, Mo...","Norwest Venture Partners, General Catalyst, Fu..."
2878,Ess Kay Fincorp,"$33,000,000",Organised Non-Banking Finance Company,Banking,NaN,Rajasthan,NaT,Rajendra Setia,"TPG, Norwest Venture Partners, Evolvence India"


### Checking for duplicated values in the dataframe

In [ ]:
# Code for checking total duplicates
df.duplicated().sum()

23

##### There was a total of 23 duplicates from the data-frame

In [ ]:
# Checking the duplicates in a data-frame
duplicated_rows = df[df.duplicated(subset=df.columns.tolist(), keep=False)]
sorted_duplicates = duplicated_rows.sort_values(by=df.columns.tolist())
sorted_duplicates.head(10)

,company_name,amount,company_activity,sector,round,head_quarter,year_founded,founders,investor
238,ASQI Advisors,$1000000,Bringing Blockchain technology intro mainstrea...,Financial Services,Pre-series A,Mumbai,2019-01-01,Swapnil Pawar,Founders Room Capital
252,ASQI Advisors,$1000000,Bringing Blockchain technology intro mainstrea...,Financial Services,Pre-series A,Mumbai,2019-01-01,Swapnil Pawar,Founders Room Capital
104,Advantage Club,$1700000,Advantage Club is India's largest employee eng...,HRTech,NaN,Mumbai,2014-01-01,"Sourabh Deorah, Smiti Bhatt Deorah","Y Combinator, Broom Ventures, Kunal Shah"
117,Advantage Club,$1700000,Advantage Club is India's largest employee eng...,HRTech,NaN,Mumbai,2014-01-01,"Sourabh Deorah, Smiti Bhatt Deorah","Y Combinator, Broom Ventures, Kunal Shah"
96,Bewakoof,$8000000,Bewakoof is a lifestyle fashion brand that mak...,Apparel & Fashion,NaN,Mumbai,2012-01-01,Prabhkiran Singh,InvestCorp
109,Bewakoof,$8000000,Bewakoof is a lifestyle fashion brand that mak...,Apparel & Fashion,NaN,Mumbai,2012-01-01,Prabhkiran Singh,InvestCorp
1535,Byju’s,500000000.0,An Indian educational technology and online tu...,EdTech,NaN,Bangalore,2011-01-01,Byju Raveendran,"Owl Ventures, Tiger Global Management"
1571,Byju’s,500000000.0,An Indian educational technology and online tu...,EdTech,NaN,Bangalore,2011-01-01,Byju Raveendran,"Owl Ventures, Tiger Global Management"
94,Curefoods,$13000000,Healthy & nutritious foods and cold pressed ju...,Food & Beverages,NaN,Bangalore,2020-01-01,Ankit Nagori,"Iron Pillar, Nordstar, Binny Bansal"
107,Curefoods,$13000000,Healthy & nutritious foods and cold pressed ju...,Food & Beverages,NaN,Bangalore,2020-01-01,Ankit Nagori,"Iron Pillar, Nordstar, Binny Bansal"


### Droping the duplicates from the data-frame